In [96]:
import pandas as pd
import re
import requests
import time
import numpy as np

读取通过uniprotKB列表检索得到的，uniref ID和KEGG的对应表，取前100,000个条目
1. 去NaN值，并且对列名重新命名
2. 拆分：对于 1个 uniref 对应多个 KEGG 的情况，

In [2]:
tmp_uniref_to_kegg = pd.read_csv("./idmapping_100000.tsv",sep="\t")

In [3]:
print(tmp_uniref_to_kegg.shape)
tmp_uniref_to_kegg.head(10)

(91490, 3)


,From,Entry,KEGG
0,A0A009E9G3,A0A009E9G3,NaN
1,A0A009EC87,A0A009EC87,NaN
2,A0A009ECP6,A0A009ECP6,NaN
3,A0A009EJT5,A0A009EJT5,NaN
4,A0A009EM74,A0A009EM74,NaN
5,A0A009EPE0,A0A009EPE0,NaN
6,A0A009EQE2,A0A009EQE2,NaN
7,A0A009EQK5,A0A009EQK5,NaN
8,A0A009ERH5,A0A009ERH5,NaN
9,A0A009ES05,A0A009ES05,NaN


In [5]:
# 去掉NaN情况
tmp_uniref_to_kegg = tmp_uniref_to_kegg[~tmp_uniref_to_kegg['KEGG'].isna()]

In [6]:
print(tmp_uniref_to_kegg.shape)
tmp_uniref_to_kegg.head(30)

(14637, 3)


,From,Entry,KEGG
224,A0A023UEV9,A0A023UEV9,shh:ShL2_00055;
227,A0A023UGA4,A0A023UGA4,shh:ShL2_00101;
253,A0A028ZLY2,A0A028ZLY2,shh:ShL2_00073;
265,A0A031FNR5,A0A031FNR5,moo:BWL13_02343;
978,A0A060AB44,A0A060AB44,vg:19685786;
979,A0A060AB89,A0A060AB89,vg:19685835;
980,A0A060ABD0,A0A060ABD0,vg:19685880;
981,A0A060AET7,A0A060AET7,vg:19685756;
982,A0A060AF10,A0A060AF10,vg:19685841;
983,A0A060AF31,A0A060AF31,vg:19685866;


In [8]:
# 去除KEGG末尾的分号，对于存在多个KEGG的条目，进行拆分

# 创建一个空的列表用于存储新的行数据
new_rows = []

for index, row in tmp_uniref_to_kegg.iterrows():
    uniref_id = row['From']
    kegg_ids = row['KEGG'].split(';') # 通过;拆分 KEGG IDs

    for kegg_id in kegg_ids:
        if kegg_id:
            new_rows.append({
                'Uniref_ID': uniref_id,
                'KEGG_ID': kegg_id
            })

# 将新的字典列表转换df
uniref_to_kegg = pd.DataFrame(new_rows)

In [9]:
print(uniref_to_kegg.shape)
uniref_to_kegg.head(30)

(15148, 2)


,Uniref_ID,KEGG_ID
0,A0A023UEV9,shh:ShL2_00055
1,A0A023UGA4,shh:ShL2_00101
2,A0A028ZLY2,shh:ShL2_00073
3,A0A031FNR5,moo:BWL13_02343
4,A0A060AB44,vg:19685786
5,A0A060AB89,vg:19685835
6,A0A060ABD0,vg:19685880
7,A0A060AET7,vg:19685756
8,A0A060AF10,vg:19685841
9,A0A060AF31,vg:19685866


In [10]:
#  查看冗余情况 按理，KEGG_ID大部分都是唯一的
uniref_to_kegg[uniref_to_kegg['KEGG_ID'].duplicated()]

,Uniref_ID,KEGG_ID
11008,H2NJB1,pon:100439869
11676,P05643-2,zma:845193
11791,P16284,hsa:5175
12050,P68363-2,hsa:10376
14947,E2JJS1,hsa:100288687


In [11]:
# 存储 新获得的 uniref_KEGG的df
uniref_to_kegg.to_csv("./new_results/uniref_to_kegg_100000.csv", index=False)

# 2. 将获取的KEGG_IDs对应到KO上，调用API，（注意，API调用，每秒只能请求3次），这里请求1次后 sleep 1秒

In [12]:
# 定义一个将提取得到的文本转换为df的函数
def webtext_to_df(x):
    import pandas as pd
    rows = x.split('\n')
    data = [row.split('\t') for row in rows]
    df = pd.DataFrame(data, columns=['org:gene','ko'])
    df.drop(df.index[-1],inplace=True)
    return df

In [15]:
# 将 keggid_list 按 100 行分块
chunk_size = 100

# 初始化存储所有请求结果的变量
all_results = []

# 循环每个分块进行请求
for i in range(0, len(uniref_to_kegg['KEGG_ID']), chunk_size):
    # 获取当前的 100 行数据
    current_chunk = uniref_to_kegg['KEGG_ID'][i:i + chunk_size]
    
    # 将列表转换为以加号（+）连接的字符串，适合 URL 请求格式
    chunk_str = '+'.join(current_chunk)
    
    # 构建请求 URL
    url = f'https://rest.kegg.jp/link/ko/{chunk_str}'
    
    try:
        # 发送 GET 请求
        response = requests.get(url)
        
        if response.status_code == 200:
            # 将请求的文本结果添加到总结果中
            all_results.append(response.text)
        else:
            print(f"请求失败！ HTTP 状态码: {response.status_code}")
        
    except requests.exceptions.RequestException as e:
        print(f"请求发生错误: {e}")
    
    # 每次请求后睡眠1秒
    time.sleep(1)

# 将所有结果汇总为一个字符串
final_result = ''.join(all_results)

# 字符转换为df 这里表示 org:gene -> ko
org_to_ko = webtext_to_df(final_result)

In [16]:
print(org_to_ko.shape)
org_to_ko.head(10)

(9744, 2)


,org:gene,ko
0,shh:ShL2_00101,ko:K00852
1,sxl:SXYLSMQ121_1778,ko:K07816
2,sxo:SXYL_01926,ko:K07816
3,sxy:BE24_02375,ko:K07816
4,sxl:SXYLSMQ121_1567,ko:K02959
5,sxo:SXYL_01645,ko:K02959
6,sxy:BE24_03470,ko:K02959
7,pfp:PFL1_04946,ko:K02980
8,pfp:PFL1_04823,ko:K03009
9,pfp:PFL1_03718,ko:K02147


In [17]:
# 存储 新获得的 org:gene到ko的df
# 去冗余
nr_org_to_ko = org_to_ko.drop(org_to_ko[org_to_ko.duplicated()].index)

org_to_ko.to_csv("./new_results/org_to_ko_9744.csv", index=False)
nr_org_to_ko.to_csv("./new_results/nr_org_to_ko_9744.csv", index=False)

# 3. 根据 ko 得到其对应的symbol上
试验过后，发现这条路会很复杂，尤其是和参考基因表相对应！

In [30]:
# 读取 ko 列表
ko_list = pd.read_csv("./ko.txt", sep='\t', names=['ko','Info'])
# 定义一个函数给 KO id 添加前缀 'ko:'
ko_list['ko'] = ko_list['ko'].apply(lambda x: f'ko:{x}')
ko_list.head()

,ko,Info
0,ko:K00001,"E1.1.1.1, adh; alcohol dehydrogenase [EC:1.1.1.1]"
1,ko:K00002,"AKR1A1, adh; alcohol dehydrogenase (NADP+) [EC..."
2,ko:K00003,hom; homoserine dehydrogenase [EC:1.1.1.3]
3,ko:K00004,"BDH, butB; (R,R)-butanediol dehydrogenase / me..."
4,ko:K00005,gldA; glycerol dehydrogenase [EC:1.1.1.6]


In [59]:
# 定义提取基因 symbol 的函数
def extract_gene_symbols(info):
    first_part = info.split(';')[0]
    # 然后按逗号分隔
    parts = first_part.split(',')
    # 使用正则表达式匹配只含大写字母、数字和 - 的 symbol
    valid_symbols = [part.strip() for part in parts if re.match(r'^[A-Z0-9-_]+$', part.strip())]
    return ';'.join(valid_symbols)

In [60]:
ko_list['gene_symbol'] = ko_list['Info'].apply(extract_gene_symbols)

In [61]:
ko_list.head(10)

,ko,Info,gene_symbol
0,ko:K00001,"E1.1.1.1, adh; alcohol dehydrogenase [EC:1.1.1.1]",
1,ko:K00002,"AKR1A1, adh; alcohol dehydrogenase (NADP+) [EC...",AKR1A1
2,ko:K00003,hom; homoserine dehydrogenase [EC:1.1.1.3],
3,ko:K00004,"BDH, butB; (R,R)-butanediol dehydrogenase / me...",BDH
4,ko:K00005,gldA; glycerol dehydrogenase [EC:1.1.1.6],
5,ko:K00006,GPD1; glycerol-3-phosphate dehydrogenase (NAD+...,GPD1
6,ko:K00007,dalD; D-arabinitol 4-dehydrogenase [EC:1.1.1.11],
7,ko:K00008,"SORD, gutB; L-iditol 2-dehydrogenase [EC:1.1.1...",SORD
8,ko:K00009,mtlD; mannitol-1-phosphate 5-dehydrogenase [EC...,
9,ko:K00010,iolG; myo-inositol 2-dehydrogenase / D-chiro-i...,


In [62]:
# 合并第二个数据框和第一个数据框，按 KO id 匹配
result_df = pd.merge(nr_org_to_ko, ko_list[['ko', 'gene_symbol']], on='ko', how='left')

In [63]:
print(result_df.shape)
result_df.head(30)

(9739, 3)


,org:gene,ko,gene_symbol
0,shh:ShL2_00101,ko:K00852,RBKS
1,sxl:SXYLSMQ121_1778,ko:K07816,
2,sxo:SXYL_01926,ko:K07816,
3,sxy:BE24_02375,ko:K07816,
4,sxl:SXYLSMQ121_1567,ko:K02959,RP-S16;MRPS16
5,sxo:SXYL_01645,ko:K02959,RP-S16;MRPS16
6,sxy:BE24_03470,ko:K02959,RP-S16;MRPS16
7,pfp:PFL1_04946,ko:K02980,RPS29
8,pfp:PFL1_04823,ko:K03009,RPABC4;RPB12;POLR2K
9,pfp:PFL1_03718,ko:K02147,ATP6B


In [64]:
# 保留 'Gene_Symbol' 列不是空字符串的行
org_to_symbol = result_df[result_df['gene_symbol'] != '']

In [65]:
print(org_to_symbol.shape)
org_to_symbol.head(30)

(5692, 3)


,org:gene,ko,gene_symbol
0,shh:ShL2_00101,ko:K00852,RBKS
4,sxl:SXYLSMQ121_1567,ko:K02959,RP-S16;MRPS16
5,sxo:SXYL_01645,ko:K02959,RP-S16;MRPS16
6,sxy:BE24_03470,ko:K02959,RP-S16;MRPS16
7,pfp:PFL1_04946,ko:K02980,RPS29
8,pfp:PFL1_04823,ko:K03009,RPABC4;RPB12;POLR2K
9,pfp:PFL1_03718,ko:K02147,ATP6B
10,pfp:PFL1_02877,ko:K02976,RPS26
11,pfp:PFL1_02890,ko:K10573,UBE2A;UBC2;RAD6A
12,spar:SPRG_06573,ko:K11254,H4


In [67]:
org_to_symbol.to_csv('./new_results/org_to_symbol.csv', index=False)

In [75]:
ref_gene_symbols = pd.read_csv('./OS_scRNA_gene_index.19264.tsv',sep='\t')
ref_gene_symbols.head(10)
gene_list = ref_gene_symbols['gene_name'].to_list()

In [83]:
uniref_to_symbol_fulllink = pd.merge(uniref_to_kegg, org_to_symbol, left_on='KEGG_ID', right_on='org:gene', how='inner')

In [84]:
uniref_to_symbol_fulllink.head()

,Uniref_ID,KEGG_ID,org:gene,ko,gene_symbol
0,A0A023UGA4,shh:ShL2_00101,shh:ShL2_00101,ko:K00852,RBKS
1,A0A060MLB4,sxl:SXYLSMQ121_1567,sxl:SXYLSMQ121_1567,ko:K02959,RP-S16;MRPS16
2,A0A060MLB4,sxo:SXYL_01645,sxo:SXYL_01645,ko:K02959,RP-S16;MRPS16
3,A0A060MLB4,sxy:BE24_03470,sxy:BE24_03470,ko:K02959,RP-S16;MRPS16
4,A0A061H4A1,pfp:PFL1_04946,pfp:PFL1_04946,ko:K02980,RPS29


In [85]:
uniref_to_symbol = uniref_to_symbol_fulllink[['Uniref_ID', 'gene_symbol']]

In [90]:
uniref_to_symbol_fulllink.to_csv('./new_results/uniref_to_symbol_fulllink.csv', index=False)
uniref_to_symbol.to_csv('./new_results/uniref_to_symbol.csv', index=False)

In [88]:
nr_uniref_to_symbol = uniref_to_symbol[~uniref_to_symbol[['Uniref_ID','gene_symbol']].duplicated()]

In [89]:
print(nr_uniref_to_symbol.shape)
nr_uniref_to_symbol.head(10)

(5376, 2)


,Uniref_ID,gene_symbol
0,A0A023UGA4,RBKS
1,A0A060MLB4,RP-S16;MRPS16
4,A0A061H4A1,RPS29
5,A0A061H6P5,RPABC4;RPB12;POLR2K
6,A0A061H8G8,ATP6B
7,A0A061H9U2,RPS26
8,A0A061HCA1,UBE2A;UBC2;RAD6A
9,A0A067CU13,H4
17,A0A068NG42,RP-S14;MRPS14
18,A0A068NGE0,RP-L14;MRPL14


In [91]:
nr_uniref_to_symbol.to_csv('./new_results/nr_uniref_to_symbol.csv', index=False)

那gene symbol去遍历基因表，
对应关系分为2种
1. gene_symbol 只有1个，直接和基因表对比。如果发现能对应上，则确定就是这个基因，如果发现symbol属于基因集中某一些元素的子集，可以通过symbol+数字对应上，则返回这些元素的集合，以;进行分隔
2. gene_symbol有多个，以;分隔，这个时候需要看存不存在一个symbol能够和基因表对应，如果有对应，则返回该基因名

In [118]:
def find_matches(value, gene_list):
    matches = []

    for gene in gene_list:
        if gene == value:
            matches.append(gene)
        elif gene.startswith(value) and re.match(r'^' + re.escape(value) + r'[A-Z]?\d+$', gene):
            matches.append(gene)

    return ';'.join(matches) if matches else ''

nr_uniref_to_symbol['hsa_gene'] = ''

for index, row in nr_uniref_to_symbol.iterrows():
    symbols = row['gene_symbol'].split(';')
    if len(symbols) == 1:
        # row['hsa_gene'] = find_matches(symbols[0], gene_list)
        nr_uniref_to_symbol.at[index, 'hsa_gene'] = find_matches(symbols[0], gene_list)
    else :
        for symbol in symbols:
            is_match = find_matches(symbol, gene_list)
            if is_match:
                nr_uniref_to_symbol.at[index, 'hsa_gene'] = is_match
        
        

/tmp/ipykernel_52349/3573488851.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nr_uniref_to_symbol['hsa_gene'] = ''


In [119]:
nr_uniref_to_symbol.head(10)

,Uniref_ID,gene_symbol,hsa_gene
0,A0A023UGA4,RBKS,RBKS
1,A0A060MLB4,RP-S16;MRPS16,MRPS16
4,A0A061H4A1,RPS29,RPS29
5,A0A061H6P5,RPABC4;RPB12;POLR2K,POLR2K
6,A0A061H8G8,ATP6B,
7,A0A061H9U2,RPS26,RPS26
8,A0A061HCA1,UBE2A;UBC2;RAD6A,UBE2A
9,A0A067CU13,H4,H4C1;H4C11;H4C12;H4C13;H4C14;H4C15;H4C2;H4C3;H...
17,A0A068NG42,RP-S14;MRPS14,MRPS14
18,A0A068NGE0,RP-L14;MRPL14,MRPL14


In [120]:
uniref_to_hsa = nr_uniref_to_symbol[nr_uniref_to_symbol['hsa_gene'] != '']

In [121]:
print(uniref_to_hsa.shape)
uniref_to_hsa.head(10)

(3925, 3)


,Uniref_ID,gene_symbol,hsa_gene
0,A0A023UGA4,RBKS,RBKS
1,A0A060MLB4,RP-S16;MRPS16,MRPS16
4,A0A061H4A1,RPS29,RPS29
5,A0A061H6P5,RPABC4;RPB12;POLR2K,POLR2K
7,A0A061H9U2,RPS26,RPS26
8,A0A061HCA1,UBE2A;UBC2;RAD6A,UBE2A
9,A0A067CU13,H4,H4C1;H4C11;H4C12;H4C13;H4C14;H4C15;H4C2;H4C3;H...
17,A0A068NG42,RP-S14;MRPS14,MRPS14
18,A0A068NGE0,RP-L14;MRPL14,MRPL14
19,A0A068NGI7,RP-L36;MRPL36,MRPL36


In [122]:
uniref_to_hsa.to_csv('./new_results/uniref_to_hsa.csv', index = False)